In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import string
import nltk
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score as accuracy
import random
import torch
import transformers
from transformers import BertTokenizer, BertModel, BertForQuestionAnswering

from collections import OrderedDict
from operator import itemgetter
from collections import Counter

import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train = pd.read_csv('/kaggle/input/sentiment-dataset-with-1-million-tweets/dataset.csv')

Let's choose only english language

In [ ]:
train = train[train['Language'] == 'en']

Now we should clean the text

In [ ]:
train['Text'] = train['Text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).str.replace('@[A-Za-z0-9]+\s?', '', regex=True)


In [ ]:
def clean_text(data):

    data = data.str.lower()
    data = data.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).str.replace('@[A-Za-z0-9]+\s?', '', regex=True)
    data = data.str.replace('[^\w\s]', '')
    data = data.str.replace('_[A-Za-z0-9]+\s?', '', regex=True)

    return data

In [ ]:
train['Text'] = clean_text(train['Text'])

Now we will tokenize Text

In [ ]:
def tokenize(text):

    tokens = nltk.word_tokenize(text)
    stop_words = nltk.corpus.stopwords.words('english')
    tokens = [i for i in tokens if (i not in stop_words)]
    tokens = [i.replace('«', '').replace('»', '').replace('--', '').replace("''", '').replace("``", '') for i in tokens]
    tokens = [i.lower() for i in tokens if i != '']

    return tokens

def tokenize_sent(text):
    sentences = [tokenize(sent) for sent in nltk.sent_tokenize(text, 'english')]
    flatten = [x for b in sentences for x in b]
    return flatten

def clean_up(lst):
    return [[w for w in sublist if w not in string.punctuation] for sublist in lst]


In [ ]:
train['tokenized'] = train['Text'].apply(tokenize_sent)

train.head(2)

,Text,Language,Label,tokenized
0,"testimony is not evidence in a court of law, s...",en,litigious,"[testimony, evidence, court, law, ,, state, fe..."
2,flagstar bank discloses a data breach that im...,en,litigious,"[flagstar, bank, discloses, data, breach, impa..."


In [ ]:
y_train = train['Label']
X_train = train['Text']

In [ ]:
!brew install wget
!wget -O GoogleNews-vectors-negative300.bin.gz "drive.google.com/u/3/uc?id=1pPPHMNjJAb82-xYFFJLz_kqcnNhmwAhW&export=download&confirm=yes"
!gzip -d GoogleNews-vectors-negative300.bin.gz n


/bin/bash: line 1: brew: command not found
--2023-11-12 13:35:37--  http://drive.google.com/u/3/uc?id=1pPPHMNjJAb82-xYFFJLz_kqcnNhmwAhW&export=download&confirm=yes
Resolving drive.google.com (drive.google.com)... 142.250.107.139, 142.250.107.100, 142.250.107.113, ...
Connecting to drive.google.com (drive.google.com)|142.250.107.139|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/u/3/uc?id=1pPPHMNjJAb82-xYFFJLz_kqcnNhmwAhW&export=download&confirm=yes [following]
--2023-11-12 13:35:37--  https://drive.google.com/u/3/uc?id=1pPPHMNjJAb82-xYFFJLz_kqcnNhmwAhW&export=download&confirm=yes
Connecting to drive.google.com (drive.google.com)|142.250.107.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1pPPHMNjJAb82-xYFFJLz_kqcnNhmwAhW&export=download&confirm=yes [following]
--2023-11-12 13:35:37--  https://drive.google.com/uc?id=1pPPHMNjJAb82-xYFFJLz_kqcnNhmwAhW&expor

In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
model = Pipeline((("vector", TfidfVectorizer()),("LR", LogisticRegression(max_iter=600))))
model.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Pipeline(steps=[('vector', TfidfVectorizer()),
                ('LR', LogisticRegression(max_iter=600))])

In [ ]:
from sklearn.metrics import accuracy_score as accuracy

In [ ]:
accuracy = (accuracy(model.predict(X_val), y_val.values))
print("Accuracy : ", accuracy)
Prediction = model.predict(X_val)

Accuracy :  0.9715141568442919


In [ ]:
print(model.predict(['you are a bad person']))

['negative']


In [ ]:
train_new1 = train[['Label', 'Text']].sample(frac=0.3, random_state = 42)

In [ ]:
train_new1['Text'] = train_new1['Text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).str.replace('@[A-Za-z0-9]+\s?', '', regex=True)


In [ ]:
def clean_text(data):

    data = data.str.lower()
    data = data.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).str.replace('@[A-Za-z0-9]+\s?', '', regex=True)
    data = data.str.replace('[^\w\s]', '')
    data = data.str.replace('_[A-Za-z0-9]+\s?', '', regex=True)

    return data

In [ ]:
train_new1['Text'] = clean_text(train_new1['Text'])

In [ ]:
labels_dict = {'positive':0,
          'negative':1,
          'uncertainty':2,
          'litigious':3,
          }

In [ ]:
train_new1['Label'] = train_new1['Label'].map(labels)

In [ ]:
train_new = train_new1[['Label', 'Text']]

In [ ]:
train_new1

,Label,Text
658880,2,"“no party nomination, with the possible except..."
927454,0,"i get asked often, “do the results really last..."
623230,0,what a incredible giveaway 👌 \n\n
387639,0,"static, coveting subtle radiance.\n\nanaphoric..."
308229,1,no health care for smokers? figure out who ca...
...,...,...
126806,1,must share moment. ignore my very bad cough. i...
857458,0,"of course, as i’m sure you know, modern medica..."
904475,1,what do you think about your h #discount #que...
273238,3,"built in a fertile agricultural area, it devel..."


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_new1.Text.values,
                                                  train_new1.Label.values,
                                                  test_size=0.15,
                                                  random_state=42)

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)


In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    X_train,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
labels_train = torch.tensor(y_train)

In [ ]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

In [ ]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                           labels_val)

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased',
                                      num_labels = 4,
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


In [ ]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup


In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
import torch
from tqdm.notebook import tqdm

for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')